In [159]:
ua = {"User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.57"}
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import pickle
import re
import requests
import time

In [160]:
url =  "https://www.nankankeiba.com"
course_d = { "浦和": "18", "船橋": "19", "大井": "20", "川崎": "21" }

def get_soup(url):
    res = requests.get(url, headers=ua)
    
    return BeautifulSoup(res.content, "html.parser")

def get_dfs(url):
    soup = get_soup(url)
    dfs = []
    if soup.find("table"):
        dfs = pd.io.html.read_html(soup.prettify())
    else:
        print(f"it's no table! {url}")
    return dfs

In [211]:
def today_url():
    base = "/program/00000000000000.do"
    soup = get_soup(url+base)

    text = soup.select_one("div.bl-left").text
    kaisai, kaisai_date = re.sub("\r|\n|\u3000", "", text).split("2022年")
    month_txt, date_txt = kaisai_date.strip("日").split("月")
    no1, no2 = kaisai.split("競馬")

    date = month_txt.rjust(2, "0") + date_txt.rjust(2, "0")
    d = { "浦和": "18", "船橋": "19", "大井": "20", "川崎": "21" }
    course = d[no1[-2:]]
    no = re.sub("\\D", "", no1).rjust(2, "0") + re.sub("\\D", "", no2).rjust(2, "0")

    return "2022" + date + course + no

def get_racename(hold: str, race: int):
    
    race_url = url + "/race_info/" + hold + str(race).rjust(2, "0") + ".do"
    soup = get_soup(race_url)
    _racename = soup.select_one("span.race-name").text
    racename = re.sub("\u3000", "", _racename)

    txs = soup.select_one("p.tx-small").text.split()[0:3]
    racetype = " ".join(txs)
    
    s = soup.select_one("div#race-data01-a")
    time = s.select_one("strong").text
    course = re.sub("\n", "", s.select_one("a").text)
    
    r = str(race) + "R"

    return r, racename, racetype, course, time

In [210]:
race = 3
hold = today_url()
race_url = url + "/race_info/" + hold + str(race).rjust(2, "0") + ".do"
dfs = get_dfs(race_url)
df = dfs[-1]
df

,枠 番,馬 番,馬名 生年月日,性齢 毛色,馬体重 増減,負担 重量,騎手名 (所属),父 母,調教師 （所属）,馬主 生産牧場,単勝 オッズ,お気に入り馬
0,1,1,セイエクシード 18.4.15(中同名),牡4 黒鹿,466 ＋3,56.0,吉井章 (大井),ゼンノロブロイ グラールホーリー,米田英世 (大井),金田成基 庄野牧場,writeOdds(1);,NaN
1,2,2,シングンスパーク 16.2.27(中同名),牡6 鹿毛,454 -1,56.0,安藤洋一 (大井),シングンオペラ エフロレス,三坂盛雄 (大井),伊坂重憲 広富牧場,writeOdds(2);,NaN
2,3,3,シントートラノスケ 18.3.20,セ4 鹿毛,434 -2,56.0,西啓太 (大井),ルースリンド ジュンナカヤマ,田中正人 (大井),中野辰三 西村牧場,writeOdds(3);,NaN
3,3,4,スケボーマン 18.5.9(中同名),牡4 黒鹿,483 ＋11,56.0,矢野貴之 (大井),ガルボ トゥファーアウェイ,朝倉実 (大井),石川秀守 白井吉美,writeOdds(4);,NaN
4,4,5,ターコイズリング 17.5.4(中同名),牝5 鹿毛,475 ＋5,54.0,的場文男 (大井),グランデッツァ シャトーモア,高野毅 (大井),堀口晴男 中川浩典,writeOdds(5);,NaN
5,4,6,マリアッチ 18.4.7(中同名),牡4 鹿毛,494 -2,55.0,仲原大生 ☆ (大井),ワールドエース ゲラウトマイウエイ,辻野豊 (大井),内藤好江 キヨタケ牧場,writeOdds(6);,NaN
6,5,7,ハプルーン 16.2.19,牡6 栗毛,441 ＋1,56.0,千田洋 (大井),フリオーソ デジタルゴールド,赤嶺亮 (大井),武仲勝 坪田信作,writeOdds(7);,NaN
7,5,8,サーストントラスト 18.4.27(中同名),牡4 黒鹿,500 ＋13,56.0,今野忠成 (川崎),キズナ パープルカフェ,立花伸 (大井),齊藤宣彰 猿倉牧場,writeOdds(8);,NaN
8,6,9,コハク 14.3.13(中同名),牡8 栗毛,474 -4,56.0,川島正太郎 (船橋),デビッドジュニア ハナガスミ,辻野豊 (大井),中村勝彦 松浦牧場,writeOdds(9);,NaN
9,6,10,ヤマショウリアン 18.5.28,牡4 鹿毛,546 ＋1,56.0,藤田凌 (大井),シンボリクリスエス テレフォトグラフ,堀千亜樹 (大井),山下勇 槇本牧場,writeOdds(10);,NaN


In [216]:
hold = today_url()
racename = get_racename(hold, 4)
_dist, _ = racename[3].split("m")
dist = re.sub(",", "", _dist)[1:]

race_url = url + "/race_info/" + hold + "04" + ".do"
soup = get_soup(race_url)
tags = soup.select("a.tx-mid")
for tag in tags:
    print(tag.text, tag.get("href"))

1200
タカミズ /uma_info/2018103320.do
プーラン /uma_info/2017104010.do
エイシンシッソウ /uma_info/2018105963.do
シルキー /uma_info/2018100355.do
フェアメーゲン /uma_info/2018102150.do
リファーブル /uma_info/2018100800.do
サンワラワラ /uma_info/2015101263.do
アールウィズ /uma_info/2018100712.do
ボーフム /uma_info/2017101687.do
ザガーロ /uma_info/2018100475.do
サクラブチャン /uma_info/2013105137.do
ムサシシニスター /uma_info/2018103422.do
セナッキー /uma_info/2018101713.do
キタノライアン /uma_info/2017104173.do
レラシオネス /uma_info/2018102191.do
コーリンジャッジ /uma_info/2018104412.do


In [222]:
n = 0
link = tags[n].get("href")
dfs = get_dfs(url + link)
print(tags[n].text)
df = dfs[-1]
if df.columns[0] == "年月日":
    print(df)

タカミズ
        年月日   場名    R           レース名       距離 天候  馬場     馬番     人気  \
0  22/03/08   大井   7R     Ｃ２(十二)(十三)    1200m    曇/良  10  番   2  人   
1  22/02/09   大井   1R       Ｃ３(五)(六)    1200m    晴/良  11  番   2  人   
2  21/06/27   阪神   6R            ３歳上    1400m    曇/良   1  番  12  人   
3  21/06/03  名古屋  10R  名古屋ＣＣ　ピーチ賞 ３歳    1400m    晴/良   1  番   3  人   
4  21/04/10   新潟   1R             ３歳    1200m    晴/良  12  番   8  人   
5  21/01/17   中京   1R             ３歳    1200m    曇/良  13  番   7  人   
6  20/12/26   阪神   1R             ２歳    1200m    晴/良  12  番   4  人   
7  20/11/28   阪神   3R             ２歳    1200m    晴/良   9  番   5  人   
8  20/07/12   阪神   1R             ２歳  1200m/芝   晴/稍重  10  番   7  人   
9  20/06/21   阪神   5R             ２歳  1600m/芝    晴/良   4  番  15  人   

            着/頭数     タイム  差/事故   上3F コーナー  通過順   体重   騎手  負担  重量  調教師  \
0   2  着  /16  頭  1:14.3   0.7  36.9     15-12  434  達城龍    54.0  荒井朋   
1   1  着  /16  頭  1:14.7   0.2  38.2       7-5  432  達城龍    54.0  荒井朋   
2  12

In [239]:
for frame in df.itertuples():
    #print(frame)
    #print("---")
#     print(frame.距離, frame._6, frame._9, frame.タイム, frame.上3F)
    dist = frame.距離
    cond = frame._6.split("/")[-1]
    order = frame._9.split()[0]
    time = frame.タイム
    l3f = frame.上3F
    print(dist, cond, order, time, l3f)

1200m 良 2 1:14.3 36.9
1200m 良 1 1:14.7 38.2
1400m 良 12 1:27.7 39.7
1400m 良 1 1:30.6 39.2
1200m 良 14 1:16.7 40.9
1200m 良 14 1:15.5 38.7
1200m 良 7 1:14.3 38.1
1200m 良 4 1:13.9 37.4
1200m/芝 稍重 9 1:13.6 38.0
1600m/芝 良 13 1:37.8 36.2


In [158]:
url = "https://keiba.rakuten.co.jp/jockey/urawa"
soup = get_soup(url)
soup.select("div.jockeyList_img")

[<div class="jockeyList_img">
 <img alt="赤津 和希" onerror="this.src='https://keiba.r10s.jp/ts_img/jockey/jockey_profile/img-profile-noimage.png?id=202107121715'; this.alt=''" src="https://keiba.r10s.jp/ts_img/jockey/jockey_profile/31266.jpg?id=202107121715"/>
 </div>,
 <div class="jockeyList_img">
 <img alt="秋元 耕成" onerror="this.src='https://keiba.r10s.jp/ts_img/jockey/jockey_profile/img-profile-noimage.png?id=202107121715'; this.alt=''" src="https://keiba.r10s.jp/ts_img/jockey/jockey_profile/30982.jpg?id=202107121715"/>
 </div>,
 <div class="jockeyList_img">
 <img alt="内田 利雄" onerror="this.src='https://keiba.r10s.jp/ts_img/jockey/jockey_profile/img-profile-noimage.png?id=202107121715'; this.alt=''" src="https://keiba.r10s.jp/ts_img/jockey/jockey_profile/30145.jpg?id=202107121715"/>
 </div>,
 <div class="jockeyList_img">
 <img alt="岡田 大" onerror="this.src='https://keiba.r10s.jp/ts_img/jockey/jockey_profile/img-profile-noimage.png?id=202107121715'; this.alt=''" src="https://keiba.r10s.jp/

In [94]:
odds_url = "https://www.nankankeiba.com/odds/202203171813041101.do"
dfs = get_dfs(odds_url)
dfs[-4]

,枠番,馬番,馬名,単勝,複勝
0,1,1,ラインブレイカー,9.2,2.1-3.4
1,2,2,クールフォルテ,5.5,1.4-2.2
2,3,3,スティールルージュ,3.6,1.2-1.7
3,4,4,ヒストリックノヴァ,5.8,1.6-2.5
4,5,5,プラチナプライド,17.1,2.4-4.0
5,6,6,エミーブレイズ,競走除外,−
6,6,7,トキノゴールド,43.9,5.6-9.8
7,7,8,ティーズハクア,62.9,6.7-11.8
8,7,9,ロマンスロード,42.4,6.1-10.8
9,8,10,ピンクプラム,153.9,14.5-25.7


In [156]:
date = "0317"
course = "浦和"
hold_code = "1304"
hold = date + course_d[course] + hold_code

race_url = url + "/program/2022" + hold + ".do"
soup = get_soup(race_url)
meta = soup.select_one("div.bl-left").text
meta = re.sub("\n|\r", "", meta)
re.sub("\u3000", " ", meta)

'第13回 浦和競馬 第4日 2022年3月17日'

In [72]:
date = "0317"
course = "18"
hold = "1304"
url = "https://www.nankankeiba.com/program/2022" + date + course + hold + ".do"
dfs = get_dfs(url)
dfs[0]

,レース,発走時刻,出走表,出走表.1,オッズ,結果,変更
0,NaN,11:10,Ｃ３４歳ウ,NaN,NaN,NaN,NaN
1,NaN,11:40,３歳(八)選抜馬ウ,NaN,NaN,NaN,NaN
2,NaN,12:10,Ｃ３(三)(四),NaN,NaN,NaN,NaN
3,NaN,12:40,桜のまち幸手賞 Ｃ２(八)(九),NaN,NaN,NaN,NaN
4,NaN,13:10,浦和８００ラウンド Ｃ２,NaN,NaN,NaN,NaN
5,NaN,13:40,浦和８００ラウンド Ｃ２,NaN,NaN,NaN,NaN
6,NaN,14:15,Ｃ３(三)(四),NaN,NaN,NaN,NaN
7,NaN,14:50,桜区の花 サクラソウ賞 Ｃ２(八)(九),NaN,NaN,NaN,NaN
8,NaN,15:25,春の夢（はるのゆめ）特別 Ｃ１(二),NaN,NaN,NaN,NaN
9,NaN,16:00,春和（しゅんわ）特別 Ｂ３(三),NaN,NaN,NaN,NaN


In [73]:
race = "11"
url = "https://www.nankankeiba.com/uma_shosai/2022" + date + course + hold + race + ".do"
res = requests.get(url, headers=ua)
soup = BeautifulSoup(res.content, "html.parser")
tables = soup.select("table")

In [74]:
tags = tables[-1].select("a[href^='/uma_info']")
tags[0].get("href")

'/uma_info/2019105488.do'

In [67]:
url = "https://www.nankankeiba.com/uma_shosai/2022" + date + course + hold + race + ".do"
dfs = get_dfs(url)
#dfs[0]

In [68]:
df = dfs[1]

In [69]:
tpls = [t for t in df.itertuples()]
for i, t in enumerate(tpls[0]):
    print(i, t)

0 0
1 1
2 1
3 リアルインパクト  ラインブレイカー  [大井] 牝3 栗毛  ラインダール  （クロフネ）  吉田勝己
4 今野忠  (川崎)  54.0  荒山勝  (大井)
5 writeOdds(1);
6 -  -
7 472  ｜  491
8 大1286稍ダ 2  --  大1427良ダ 1  --  --
9 1  1  0  0  --  1  0  0  1  --  --
10 --  --  2  2  0  1  --  2  2  0  1
11 1  大井22.1.25 良内ダ 1600  ’２２桃花賞  15頭4番6人 今野忠 54.0  491k クールフォルテ  1427 (0.1) 1-1-1-1 39.7
12 11  大井21.12.31 良内ダ 1600  東京２歳優駿牝馬（ＳI）  16頭3番10人 今野忠 54.0  494k スピーディキック  1446 (1.8) 5-6-3-4 41.2
13 2  大井21.11.4 稍外ダ 1400  ２歳 はくたか特別  16頭8番4人 本橋孝 54.0  482k ロマンスロード  1286 (0.2) 4-4-3 38.7
14 1  大井21.7.1 不外ダ 1400  ２歳７２．５  9頭5番1人 藤田凌 54.0  476k コブラ  1290 (1.5) 1-1-1 39.7
15 2  大井21.6.11 良外ダ 1200  ２歳新馬  6頭3番2人 藤田凌 54.0  472k ママママカロニ  1145 (0.4) 2-2 37.3


In [24]:
tpls[0][11]

'3  浦和22.2.21 良外ダ 1400  ３歳(三)  9頭4番7人 町田直 54.0  455k コンアモーレ  1326 (1.5) 2-2-2-3 40.8'

In [78]:
url = "https://keiba.rakuten.co.jp/jockey/urawa"
res = requests.get(url, headers=ua)
soup = BeautifulSoup(res.content, "html.parser")
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="ja" prefix="og: http://ogp.me/ns# fb: http://www.facebook.com/2008/fbml mixi: http://mixi-platform.com/ns# gr: http://gree.jp/ns">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="浦和所属騎手一覧 ：楽天競馬" name="title"/>
<meta content="浦和所属騎手一覧では、浦和競馬場に所属している騎手をご紹介します。" name="description"/>
<meta content="浦和,浦和競馬場,騎手一覧,ジョッキー一覧,所属一覧" name="keywords"/>
<meta content="ja" name="language"/>
<meta content="width=1020" name="viewport"/>
<meta content="浦和所属騎手一覧 ：楽天競馬" property="og:title"/>
<meta content="浦和所属騎手一覧では、浦和競馬場に所属している騎手をご紹介します。" property="og:description"/>
<meta content="楽天競馬" property="og:site_name"/>
<meta content="https://keiba.r10s.jp/img/ogp/ogp_jockey_list.jpg?id=cfc1d6ddbb6f6a3bac31" property="og:image"/>
<meta content="website" property="og:type"/>
<meta content="277381235708

In [85]:
tags = soup.select(".jockeyList_img img")
tags[0].get("src")

'https://keiba.r10s.jp/ts_img/jockey/jockey_profile/31266.jpg?id=202107121715'